# Multi State Discrimination

In this tutorial we show how to set up an experiment to discriminate between the first $n$ energy states. The experiment will create $n$ circuits that prepare, respectively, the states $|0\rangle, \cdots, |n-1\rangle$. The measured data is then used to train a discriminator.

Here we will take $n=3$ and thus discriminate between $|0\rangle, |1\rangle$ and $|2\rangle$.

## Calibrate schedules

The first step is to calibrate the pulse schedules that take the energy level $|i\rangle$ to $|i+1\rangle$. These schedules will be later used to build the $n$ circuits for the experiment.

In [1]:
import pandas as pd

from qiskit import IBMQ, pulse

from qiskit.circuit import Parameter
from qiskit_experiments.calibration_management import Calibrations
from qiskit_experiments.library import EFRoughXSXAmplitudeCal

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibm_perth')

In [3]:
qubit = 0

In [4]:
# get schedule for |1> -> |2>
cals = Calibrations.from_backend(backend)

props = backend.properties()

anharm = props.qubit_property(qubit)['anharmonicity'][0]

chan = Parameter('ch0')
amp_x = Parameter('amp')
amp_sx = Parameter('amp')

with pulse.build(name='x12') as x12:
    pulse.shift_frequency(anharm, pulse.DriveChannel(chan))
    pulse.play(pulse.Gaussian(160, amp_x, sigma=40), pulse.DriveChannel(chan))
    pulse.shift_frequency(-anharm, pulse.DriveChannel(chan))
    
with pulse.build(name='sx12') as sx12:
    pulse.shift_frequency(anharm, pulse.DriveChannel(chan))
    pulse.play(pulse.Gaussian(160, amp_sx, sigma=40), pulse.DriveChannel(chan))
    pulse.shift_frequency(-anharm, pulse.DriveChannel(chan))
    
cals.add_schedule(x12, qubits=qubit, num_qubits=1)
cals.add_schedule(sx12, qubits=qubit, num_qubits=1)
cals.add_parameter_value(0.5, "amp", qubits=qubit, schedule=x12)
cals.add_parameter_value(0.25, "amp", qubits=qubit, schedule=sx12)

In [5]:
# Create Rabi experiment
exp = EFRoughXSXAmplitudeCal(qubit, cals, backend=backend)
exp_data = exp.run()

Job was cancelled before completion [Job ID: 636536c3b54402a783cb5e81]


In [6]:
exp_data.status()

<ExperimentStatus.DONE: 'experiment jobs and analysis have successfully run'>

In [7]:
pd.DataFrame(**cals.parameters_table(qubit_list=[0,()]))

,parameter,qubits,schedule,value,group,valid,date_time,exp_id
0,drive_freq,"(0,)",None,5.157545e+09+0.000000e+00j,default,True,2022-11-02 09:11:29.225310+0100,None
1,amp,"(0,)",x12,1.239369e-01+0.000000e+00j,default,True,2022-11-02 09:40:41.621000+0100,7e80f8e2-3a38-4c4a-ae7f-9cb71a894ca2
2,meas_freq,"(0,)",None,7.163116e+09+0.000000e+00j,default,True,2022-11-02 09:11:29.225310+0100,None
3,amp,"(0,)",sx12,6.196845e-02+0.000000e+00j,default,True,2022-11-02 09:40:41.621000+0100,7e80f8e2-3a38-4c4a-ae7f-9cb71a894ca2


## Discriminator experiment

Now that the pulse schedules are calibrated we can build the multi state discriminator experiment. The calibrated schedules are used as an initialisation argument for the experiment.

In [8]:
from qiskit_experiments.library import MultiStateDiscrimination
exp_disc = MultiStateDiscrimination(qubit, n_states=3, backend=backend, schedules={'x12':cals.get_schedule('x12', qubit)})

The circuits preparing the different energy levels are of the form:

In [9]:
# circuit preparing |0>
exp_disc.circuits()[0].draw()

░ ┌─┐
   q: ─░─┤M├
       ░ └╥┘
meas: ════╩═

In [10]:
# circuit preparing |1>
exp_disc.circuits()[1].draw()

┌───┐ ░ ┌─┐
   q: ┤ X ├─░─┤M├
      └───┘ ░ └╥┘
meas: ═════════╩═

In [11]:
# circuit preparing |2>
exp_disc.circuits()[2].draw()

┌───┐┌─────┐ ░ ┌─┐
   q: ┤ X ├┤ x12 ├─░─┤M├
      └───┘└─────┘ ░ └╥┘
meas: ════════════════╩═

In [12]:
exp_data_disc = exp_disc.run()

In [ ]:
exp_disc.analysis_results

In [ ]:
exp_data_disc.figure(0)